In [100]:
import random
import matplotlib.pyplot as plt
import numpy as np
import math
class Agent:

    def __init__(self):
        self.money = 0
        self.class_type = 'middle'

    def consume(self, seller, amount):
          self.money -= amount
          seller.money += amount

In [101]:
import cupy as cp

def roulette_wheel_selection_gpu(probs):
    # Ensure the input is a CuPy array
    if not isinstance(probs, cp.ndarray):
        probs = cp.array(probs)

    cumulative_probs = cp.cumsum(probs)  # Use CuPy's cumsum
    r = cp.random.random() * cumulative_probs[-1]

    index = cp.searchsorted(cumulative_probs, r)
    return index.item()


In [125]:
class Simulation:
    def __init__(self, num_agents, num_time_steps):
        self.num_agents = num_agents
        self.num_time_steps = num_time_steps
        self.agents = [Agent() for _ in range(num_agents)]
        self.total_tax = 0
        self.update_thresholds()
        self.update_wealth()
        self.segregate_agents()
        self.trade_probs = cp.zeros((self.num_agents, self.num_agents))
        self.initialize_trade_probabilities()
        self.rich_agent_wealth = []
        self.middle_agent_wealth = []
        self.poor_agent_wealth = []
        self.rich_agent_classes = []
        self.middle_agent_classes = []
        self.poor_agent_classes = []

    def update_wealth(self):
        # Generate random wealth values based on specified percentages
        wealth_values = [random.uniform(0.1, 3) for _ in range(int(0.5 * self.num_agents))] + [random.uniform(3, 12) for _ in range(int(0.4 * self.num_agents))] + [random.uniform(12, 50) for _ in range(int(0.1 * self.num_agents))]
        random.shuffle(wealth_values)

        # Update agent wealth values
        for i, agent in enumerate(self.agents):
            agent.money = wealth_values[i]


    def calculate_tax(self, wealth):
        scaled_slab_1 = 3
        scaled_slab_2 = 6
        scaled_slab_3 = 9
        scaled_slab_4 = 12
        scaled_slab_5 = 15

        if wealth <= scaled_slab_1:
            return 0
        elif wealth <= scaled_slab_2:
            return 0.05 * (wealth - scaled_slab_1)
        elif wealth <= scaled_slab_3:
            return 0.1 * (wealth - scaled_slab_2) + 0.05 * (scaled_slab_2 - scaled_slab_1)
        elif wealth <= scaled_slab_4:
            return 0.15 * (wealth - scaled_slab_3) + 0.05 * (scaled_slab_2 - scaled_slab_1) + 0.1 * (scaled_slab_3 - scaled_slab_2)
        elif wealth <= scaled_slab_5:
            return 0.2 * (wealth - scaled_slab_4) + 0.05 * (scaled_slab_2 - scaled_slab_1) + 0.1 * (scaled_slab_3 - scaled_slab_2) + 0.15 * (scaled_slab_4 - scaled_slab_3)
        else:
            return 0.3 * (wealth - scaled_slab_5) + 0.05 * (scaled_slab_2 - scaled_slab_1) + 0.1 * (scaled_slab_3 - scaled_slab_2) + 0.15 * (scaled_slab_4 - scaled_slab_3) + 0.2 * (scaled_slab_5 - scaled_slab_4)

    def calculate_trade_prob(self, agent1, agent2):
        wealth_diff = cp.abs(agent1.money - agent2.money)
        return cp.exp(-wealth_diff)

    def initialize_trade_probabilities(self):
        # Initialize the matrix with trade probabilities for all pairs of agents
        for i in range(self.num_agents):
            for j in range(i + 1, self.num_agents):
                trade_prob = self.calculate_trade_prob(self.agents[i], self.agents[j])
                self.trade_probs[i, j] = trade_prob
                self.trade_probs[j, i] = trade_prob  # Symmetric matrix

    def update_trade_probabilities(self, agent1_index, agent2_index):
        # Update probabilities involving agent1 and agent2 only
        for i in range(self.num_agents):
            if i != agent1_index:
                self.trade_probs[agent1_index, i] = self.calculate_trade_prob(self.agents[agent1_index], self.agents[i])
                self.trade_probs[i, agent1_index] = self.trade_probs[agent1_index, i]  # Symmetric update

            if i != agent2_index:
                self.trade_probs[agent2_index, i] = self.calculate_trade_prob(self.agents[agent2_index], self.agents[i])
                self.trade_probs[i, agent2_index] = self.trade_probs[agent2_index, i]

    def update_agents(self):
        agent1_index = np.random.randint(0, self.num_agents)
        agent1 = self.agents[agent1_index]

        # Select the second agent using roulette wheel selection
        agent_probs = self.trade_probs[agent1_index]
        agent2_index = roulette_wheel_selection_gpu(agent_probs)
        agent2 = self.agents[agent2_index]

        # Calculate transaction amount
        transaction_amount = 0.075 * (agent1.money * agent2.money) / (agent1.money + agent2.money)
        agent1.consume(agent2, transaction_amount)

        # Update trade probabilities
        self.update_trade_probabilities(agent1_index, agent2_index)


    def deduct_tax(self):
        for agent in self.agents:
            tax = self.calculate_tax(agent.money)
            agent.money -= tax
            self.total_tax += tax

    def redist(self):
        add_val = self.total_tax / self.num_agents
        for agent in self.agents:
            agent.money += add_val
        self.total_tax = 0

    def update_thresholds(self):
        wealths = [agent.money for agent in self.agents]
        sorted_wealths = np.sort(wealths)

        self.lower_threshold = np.percentile(sorted_wealths, 50)  # 50th percentile
        self.upper_threshold = np.percentile(sorted_wealths, 90)  # 90th percentile

    def segregate_agents(self):
        for agent in self.agents:
            if agent.money <= self.lower_threshold:
                agent.class_type = 'poor'
            elif agent.money >= self.upper_threshold:
                agent.class_type = 'rich'
            else:
                agent.class_type = 'middle'

    def select_and_track_agents(self):
        random.shuffle(self.agents)
        for agent in self.agents:
            if agent.class_type == 'rich':
                self.rich_agent = agent
                break

        for agent in self.agents:
            if agent.class_type == 'middle':
                self.middle_agent = agent
                break

        for agent in self.agents:
            if agent.class_type == 'poor':
                self.poor_agent = agent
                break

        self.rich_agent_wealth.append(self.rich_agent.money)
        self.middle_agent_wealth.append(self.middle_agent.money)
        self.poor_agent_wealth.append(self.poor_agent.money)

        self.rich_agent_classes.append(self.rich_agent.class_type)
        self.middle_agent_classes.append(self.middle_agent.class_type)
        self.poor_agent_classes.append(self.poor_agent.class_type)

    def record_agent_classes(self):
        self.rich_agent_classes.append(self.rich_agent.class_type)
        self.middle_agent_classes.append(self.middle_agent.class_type)
        self.poor_agent_classes.append(self.poor_agent.class_type)

    def run_simulation(self):
        for _ in range(self.num_time_steps):
            self.update_agents()
        self.deduct_tax()
        self.redist()

In [138]:
def gini_coefficient(wealths):
    n = len(wealths)
    if n == 0:
        return 0

    # Step 1: Sort the wealth list
    sorted_wealths = np.sort(wealths)

    # Step 2: Calculate the mean absolute difference
    diff_matrix = np.abs(sorted_wealths[:, None] - sorted_wealths)
    mean_abs_diff = np.mean(diff_matrix)

    # Step 3: Compute the mean wealth
    mean_wealth = np.mean(sorted_wealths)

    # Step 4: Compute the Gini coefficient
    gini_index = mean_abs_diff / (2 * mean_wealth)
    return gini_index

In [ ]:
num_agents = 100
num_time_steps = 1000
gini_indices= []
# Initialize the simulation
sim = Simulation(num_agents, num_time_steps)

# Run simulation for a total of 500 iterations, segregating after each redistribution
for i in range(100):
    sim.run_simulation()
    if i % 50 == 0:  # Update thresholds every 50 iterations
        sim.update_thresholds()
    sim.segregate_agents()
    if i == 1:  # Select agents to track after initial segregation
        sim.select_and_track_agents()
    elif i>1:
        # Record wealth and class of the tracked agents
        sim.rich_agent_wealth.append(sim.rich_agent.money)
        sim.middle_agent_wealth.append(sim.middle_agent.money)
        sim.poor_agent_wealth.append(sim.poor_agent.money)

        sim.record_agent_classes()
    current_wealths = [agent.money for agent in sim.agents]
    gini_index = gini_coefficient(current_wealths)
    gini_indices.append(gini_index)
    print("Iteration ", i + 1, " complete")

# Plot the wealth of the tracked agents
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(sim.rich_agent_wealth, label='Rich Agent')
plt.plot(sim.middle_agent_wealth, label='Middle Agent')
plt.plot(sim.poor_agent_wealth, label='Poor Agent')
plt.xlabel('Iteration')
plt.ylabel('Wealth')
plt.legend()


# plt.show()
# # Analyze class changes
# def analyze_class_changes(agent_classes, agent_label):
#     changes = 0
#     for i in range(1, len(agent_classes)):
#         if agent_classes[i] != agent_classes[i - 1]:
#             changes += 1
#     print(f"{agent_label} changed class {changes} times.")

# analyze_class_changes(sim.rich_agent_classes, "Rich Agent")
# analyze_class_changes(sim.middle_agent_classes, "Middle Agent")
# analyze_class_changes(sim.poor_agent_classes, "Poor Agent")

Iteration  1  complete
Iteration  2  complete
Iteration  3  complete
Iteration  4  complete
Iteration  5  complete
Iteration  6  complete
Iteration  7  complete
Iteration  8  complete
Iteration  9  complete
Iteration  10  complete
Iteration  11  complete
Iteration  12  complete
Iteration  13  complete
Iteration  14  complete
Iteration  15  complete
Iteration  16  complete
Iteration  17  complete
Iteration  18  complete
Iteration  19  complete
Iteration  20  complete
Iteration  21  complete
Iteration  22  complete
Iteration  23  complete
Iteration  24  complete
Iteration  25  complete
Iteration  26  complete
Iteration  27  complete
Iteration  28  complete
Iteration  29  complete
Iteration  30  complete
Iteration  31  complete
Iteration  32  complete
Iteration  33  complete
Iteration  34  complete
Iteration  35  complete
Iteration  36  complete


In [ ]:
# Plot the Gini coefficient over iterations
plt.subplot(1, 2, 2)
plt.plot(gini_indices, label='Gini Index')
plt.xlabel('Iteration')
plt.ylabel('Gini Index')
plt.legend()


In [128]:
import cProfile

cProfile.run('sim.run_simulation()')


         2605106 function calls in 14.714 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1000    0.001    0.000    0.001    0.000 <ipython-input-100-2d00f837f1ef>:11(consume)
     1000    0.035    0.000    0.252    0.000 <ipython-input-101-579aa026a507>:9(roulette_wheel_selection_gpu)
        1    0.002    0.002   14.713   14.713 <ipython-input-125-fbb76a36d8bb>:147(run_simulation)
      100    0.000    0.000    0.000    0.000 <ipython-input-125-fbb76a36d8bb>:29(calculate_tax)
   198000    8.171    0.000    8.171    0.000 <ipython-input-125-fbb76a36d8bb>:49(calculate_trade_prob)
     1000    5.684    0.006   14.421    0.014 <ipython-input-125-fbb76a36d8bb>:61(update_trade_probabilities)
     1000    0.018    0.000   14.712    0.015 <ipython-input-125-fbb76a36d8bb>:72(update_agents)
        1    0.000    0.000    0.000    0.000 <ipython-input-125-fbb76a36d8bb>:89(deduct_tax)
        1    0.000    0.000    0.000    0.00